In [68]:
## *Heroes of Pymoli*

# dependencies and setup
import pandas as pd
import numpy as np

# Raw data file
file_to_load = "../Resources/purchase_data.csv"

# Read purchasing file and store into pandas data frame
purchase_data = pd.read_csv(file_to_load)

# snippet of file
purchase_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [69]:
# Player count - display total numbers of players
player_count = len(purchase_data["SN"].unique())
player_count

576

In [73]:
## Purchasing analysis (total)

# number of unique items
unique_items = purchase_data.drop_duplicates(['Item ID'], keep = 'last')

# total revenue
total_revenue = round(purchase_data['Price'].sum(),2)

# total number of purchases
total_purchases = purchase_data['Price'].count()

# average purchase price
avg_purchase_price = round(total_revenue/total_purchases,2)

#create df
purchase_analysis = pd.DataFrame([{
    "No. of Unique Items": unique_items,
    "Avg. Purchase Price": avg_purchase_price,
    "Total Purchases": total_purchases,
    "Total Revenue": total_revenue
}])
purchase_analysis

#format table

,Avg. Purchase Price,No. of Unique Items,Total Purchases,Total Revenue
0,3.05,Purchase ID SN Age Gender...,780,2379.77


In [13]:
## Gender demographics

# create player count by finding unique players
unique_players = purchase_data.drop_duplicates(["SN"], keep ="last")

# percentage and count of male, female, and non-disclose players
gender_counts = unique_players["Gender"].value_counts().reset_index()
gender_counts["% of Players"] = gender_counts["Gender"]/player_count * 100

#remove index and reformat table
gender_counts



,index,Gender,% of Players
0,Male,484,84.027778
1,Female,81,14.062500
2,Other / Non-Disclosed,11,1.909722


In [80]:
## Purchasing analysis (Gender)

# Purchase count
purchase_cnt_gender = pd.DataFrame(purchase_data.groupby("Gender")["Gender"].count())

# Total purchase value
total_purchase_cnt_gender = pd.DataFrame(purchase_data.groupby("Gender")["Price"].sum())

# Average purchase price
avg_purchase_price['Average Purchase Price'] = avg_purchase_price['Total Purchase Value'/avg_purchase_price['# of Purchases']]

# Normalized totals
purchase_gender_merge = pd.merge(purchase_cnt_gender, total_purchase_cnt_gender, left_index = True, right_index = True)
purchase_gender_merge["Normalized Totals"] = purchase_gender_merge["Total Purchase Value"]/purchase_gender_merge["# of Purchase"]

IndexError: invalid index to scalar variable.

In [90]:
## Age demographics

# age bins
purchase_data.loc[(purchase_data['Age'] < 10), 'age_bin'] = "< 10"
purchase_data.loc[(purchase_data['Age'] >= 10) & (purchase_data['Age'] <= 14), 'age_bin'] = "10 - 14"
purchase_data.loc[(purchase_data['Age'] >= 10) & (purchase_data['Age'] <= 14), 'age_bin'] = "15 - 19"
purchase_data.loc[(purchase_data['Age'] >= 10) & (purchase_data['Age'] <= 14), 'age_bin'] = "20 - 24"
purchase_data.loc[(purchase_data['Age'] >= 10) & (purchase_data['Age'] <= 14), 'age_bin'] = "25 - 29"
purchase_data.loc[(purchase_data['Age'] >= 10) & (purchase_data['Age'] <= 14), 'age_bin'] = "30 - 34"
purchase_data.loc[(purchase_data['Age'] >= 10) & (purchase_data['Age'] <= 14), 'age_bin'] = "35 - 39"
purchase_data.loc[(purchase_data['Age'] >= 40), 'age_bin'] = "> 40"

# Purchase count
purchase_count_age = pd.DataFrame(purchase_data.groupby('age_bin')['SN'].count())
                  
# Average purchase price
avg_price_age = pd.DataFrame(purchase_data.groupby('age_bin')['Price'].sum())
                  
# Total purchase value
total_purchases_age = pd.DataFrame(purchase_data.groupby('age_bin')['Price'].sum())

# Normalized totals
age_merge = pd.merge(purchase_data, avg_price_age, left_index = True, right_index = True).merge(total_purchases_age, left_index = True, right_index = True)
age_merge.rename(columns = {"SN_x": "# of Purchases", "Price_x": "Average Purchase Price", "Price_y": "Total Purchase Value", "SN_y": "# of Purchasers"}, inplace = True)
age_merge['Normalized Totals'] = age_merge['Total Purchase Value']/age_merge['# of Purchases']

KeyError: '# of Purchases'

In [110]:
## Top Spenders

# SN
purchases_by_SN = pd.DataFrame(purchase_data.groupby('SN')['Price'].sum())
number_of_purchase_by_SN = pd.DataFrame(purchase_data.groupby('SN')['Price'].count())
avg_purchase_by_SN = pd.DataFrame(purchase_data.groupby('SN')['Price'].mean())

# Top 5 spenders
merge_Top5_spenders = pd.merge(purchases_by_SN, number_of_purchase_by_SN, left_index = True, right_index = True).merge(avg_purchase_by_SN, left_index = True, right_index = True)
merge_Top5_spenders.rename(columns = {'Price_x': 'Total Purchase Value', 'Price_y': 'Purchase Count', 'Price': 'Avg Purchase Price'}, inplace = True)
merge_Top5_spenders = merge_Top5_spenders.head()
merge_Top5_spenders

,Total Purchase Value,Purchase Count,Avg Purchase Price
SN,,,
Adairialis76,2.28,1,2.28
Adastirin33,4.48,1,4.48
Aeda94,4.91,1,4.91
Aela59,4.32,1,4.32
Aelaria33,1.79,1,1.79


In [120]:
## Most popular items

top5_items = pd.DataFrame(purchase_data.groupby('Item ID')['Item ID'].count())
top5_items.sort_values('Item ID', ascending = False, inplace = True)
top5_items = top5_items.iloc[0:5][:]
top5_items_total = pd.DataFrame(purchase_data.groupby('Item ID')['Price'].sum())
top5_items = pd.merge(top5_items, top5_items_total, left_index = True, right_index = True)
top5_items_merge = top5_items_merge[['Item ID', 'Item Name', 'Item ID_x', 'Price_y', 'Price_x']]
top5_items_merge.rename(columns = {'Item ID': 'Purchase Count', 'Price_y': 'Item Price', 'Price_x': 'Total Purchase Value'}, inplace=True)
top5_items_merge

# display in table
# Item ID
# Item name
# Purchase count
# Item price
# Total purchase value

/anaconda3/envs/PythonData=3.6/lib/python3.6/site-packages/ipykernel/__main__.py:4: FutureWarning: 'Item ID' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version


NameError: name 'top5_items_merge' is not defined

In [121]:
## Most profitable items

top5_profitable = pd.DataFrame(purchase_data.groupby('Item ID')['Price'].sum())
top5_profitable.sort_values('Price', ascending = False, inplace = True)
top5_profitable = top5_profitable.iloc[0:5][:]
purchase_cnt_profit = pd.DataFrame(purchase_data.groupby('Item Id')['Item ID'].count())
top5_profit = pd.merge(top5_profit, purchase_cnt_profit, left_index = True, right_index = True)
top5_profit_merge = pd.merge[['Item ID', 'Item Name']]
top5_profit_merge

# display in table
    # Item ID
    # Item name
    # Purchase count
    # Item price
    # Total purchase value

KeyError: 'Item Id'